# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
weather_csv = "../WeatherPy/weatherOutput.csv"
weather_df = pd.read_csv(weather_csv)
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Los Llanos de Aridane,20,ES,1580239422,72,28.66,-17.92,66.20,6.93
1,College,20,US,1580239533,74,64.86,-147.80,-17.00,3.47
2,Port Keats,66,AU,1580239534,84,-14.25,129.55,85.71,15.26
3,Cape Town,75,ZA,1580239502,82,-33.93,18.42,71.01,12.75
4,Bluff,86,NZ,1580239534,80,-46.60,168.33,61.00,5.77
...,...,...,...,...,...,...,...,...,...
582,Woodward,90,US,1580239454,96,36.43,-99.39,33.80,13.87
583,Mount Gambier,90,AU,1580239565,93,-37.83,140.77,50.00,1.12
584,Chernyshevskiy,91,RU,1580239607,90,63.01,112.47,3.78,4.97
585,Barra do Corda,59,BR,1580239608,84,-5.50,-45.25,83.10,2.73


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
# configure gmaps, access maps with unique API key
gmaps.configure(api_key=g_key)

In [41]:
#use Lat and Lng as Locations and Humidity as the weight
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
weight = weather_df["Humidity"]
# Drop NaN values
weather_df = weather_df.dropna()
center_point = locations.median()
center_point



Lat    20.88
Lng    23.49
dtype: float64

In [42]:
# Plot Heatmap
fig = gmaps.figure(center = center_point, zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

  ![hotel map](Images/hotel_map.png)

In [43]:
#* Narrow down the DataFrame to find your ideal weather condition. For example:
 # A max temperature lower than 80 degrees but higher than 70.
 # Wind speed less than 10 mph.
 # Zero cloudiness.
ideal_df = weather_df[(weather_df['Max Temp']<80) & (weather_df['Max Temp']>70) & (weather_df['Wind Speed']<10) & (weather_df['Cloudiness']==0)]
ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
259,Broken Hill,0,AU,1580239566,21,-31.95,141.43,71.60,8.05
302,Alappuzha,0,IN,1580239572,88,9.49,76.33,75.00,5.01
322,Aliwal North,0,ZA,1580239574,17,-30.69,26.71,72.79,4.36
353,Sorong,0,ID,1580239578,97,-0.88,131.25,73.54,2.04
437,Celestun,0,MX,1580239589,79,20.87,-90.40,74.26,7.96


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
#organize columns and add "Hotel" column
ideal_df["Hotel Name"] = ""
ideal_df= ideal_df[["Hotel Name","City", "Country", "Date","Max Temp","Cloudiness", "Humidity", "Wind Speed", "Lat", "Lng"]]

#copy df
hotel_df = ideal_df.copy()
hotel_df

C:\Users\SUV\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Hotel Name,City,Country,Date,Max Temp,Cloudiness,Humidity,Wind Speed,Lat,Lng
259,,Broken Hill,AU,1580239566,71.60,0,21,8.05,-31.95,141.43
302,,Alappuzha,IN,1580239572,75.00,0,88,5.01,9.49,76.33
322,,Aliwal North,ZA,1580239574,72.79,0,17,4.36,-30.69,26.71
353,,Sorong,ID,1580239578,73.54,0,97,2.04,-0.88,131.25
437,,Celestun,MX,1580239589,74.26,0,79,7.96,20.87,-90.40


In [45]:
# set up variables to hold lists
# set up lists to hold reponse info (will be used for df)...city already done in previous code


# set up a parameters dictionary
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterrate through df
for index, row in hotel_df.iterrows():
    # get coordinates from df
    lat = row['Lat']
    lng = row['Lng']
    #update location param
    params['location'] = f"{lat},{lng}"
    # run a request using our params dictionary
    hotel_response = requests.get(base_url, params=params)

     # convert to json
    hotel_response = hotel_response.json()
    
 #   print(f"{params['location'][2]}")
 #   print(f"Closest hotel is {results[4]['name']}.")       
    hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
        
#    except (IndexError):
 #       print("Missing field/result... skipping.")


In [46]:
hotel_df

,Hotel Name,City,Country,Date,Max Temp,Cloudiness,Humidity,Wind Speed,Lat,Lng
259,Royal Exchange Hotel,Broken Hill,AU,1580239566,71.60,0,21,8.05,-31.95,141.43
302,Kerala Luxury Houseboat Alleppey,Alappuzha,IN,1580239572,75.00,0,88,5.01,9.49,76.33
322,Riverside Lodge & Conference Centre,Aliwal North,ZA,1580239574,72.79,0,17,4.36,-30.69,26.71
353,Swiss-Belhotel Sorong,Sorong,ID,1580239578,73.54,0,97,2.04,-0.88,131.25
437,Hotel San Julio,Celestun,MX,1580239589,74.26,0,79,7.96,20.87,-90.40


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
#figure_layout = {
#    'width': '600px',
#    'height': '500px',
#    'border': '1px solid black',
#    'padding': '1px',
#    'margin': '0 auto 0 auto'
#}
fig = gmaps.figure(center = center_point, zoom_level = 2)

# Create heat layer
#heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
#                                 dissipating=False, max_intensity=10,
#                                 point_radius=1)

#create hotel markers
hotel_markers = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index,row in hotel_df.iterrows()]
)

# Display Map
fig.add_layer(hotel_markers)
fig

Figure(layout=FigureLayout(height='420px'))